In [1]:
import pandas as pd
import numpy as np
import re
import re
from readabilty import Readability

In [3]:
df = pd.read_feather("PATH/TO/ALL/FILES")
df.info()

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Danny\AppData\Local\Temp\3\ipykernel_17044\476235314.py:3: SyntaxWarning: invalid escape sequence '\d'
  updated = pd.read_feather("EDGAR-CORPUS\data\edgar\datasets\item7_EDGAR-CORPUS_2021-2023.feather")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235819 entries, 0 to 235818
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   filename          235819 non-null  object 
 1   cik               235819 non-null  object 
 2   sic               233892 non-null  object 
 3   filing_date       235819 non-null  object 
 4   period_of_report  235819 non-null  object 
 5   fog_score_item7   204671 non-null  float64
 6   item_7            235819 non-null  object 
 7   num_words         231127 non-null  float64
 8   year              235819 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 16.2+ MB


In [4]:
df = df[df.year.between(1996,2022)]
df.drop_duplicates(subset=["item_7","year"], keep="first",inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
print(df.fog_score_item7.max())
print(df.fog_score_item7.min())
print(df.fog_score_item7.mean())
print(df.fog_score_item7.std())

190.3002169197397
3.257142857142857
18.486239574653997
2.450177710071397


In [9]:
df = df[df.fog_score_item7.between(13,24)]

In [11]:
# recalculate fog for better accuracy 

In [13]:
def find_item_7a_8(text):
    try:
        item8 = re.search(r"(?<!see)(?<!in)\s+item 8",text, flags=re.I)
        item9 = re.search(r"(?<!see)(?<!in)\s+item 9",text, flags=re.I)
        if item8:
               return text[:re.search(r"(?<!see)(?<!in)\s+item 8",text, flags=re.I).span()[0]]
        elif item9:
                return text[:re.search(r"(?<!see)(?<!in)\s+item 9",text, flags=re.I).span()[0]]
        else:
                return text[:re.search(r"(?<!see)(?<!in)\s+item 7A",text, flags=re.I).span()[0]]
    except: return text
    """negative lookbehined to find item 8|9 that not preceeded by the word see or in.
        the logic for not using the bitwise or | is the regular expression engine will try to match item 8 if it is not preceeded by see or in.
        The thing is that, if it is preceeded by an in, it is not preceeded by see, so the first pattern matches. 
        If it is preceeded by see, it is not preceeded by in, so the second pattern version matches. 
        In either way one of those patterns will match since in and see are mutually exclusive."""

In [15]:
def length(text):
    if text:
        return len(re.compile(r"\b\S+\b").findall(text))

In [17]:
import dask.dataframe as dd
import dask
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [21]:
def compute_readability(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return None, None, None,None, 0

    processed_text = find_item_7a_8(text)
    word_count = length(processed_text)

    if word_count is None:
        word_count = 0

    if word_count < 50:
        return None, None, None, word_count

    r = Readability(processed_text)

    try:
        ari = r.ari()[0].score
        fog = r.gunning_fog().score
        fk = r.flesch_kincaid().score
        lw = r.linsear_write().score
    except:
        ari, fog, fk, lw, word_count = None, None, None, None, None

    return ari, fog, fk, lw, word_count

@dask.delayed
def process_row(row):
    return compute_readability(row["item_7"])

In [ ]:
# this will take about 25 minutes ti run 
ddf = dd.from_pandas(df, npartitions=35)
readability_results = dask.compute([process_row(row) for _, row in df.iterrows()], scheduler="processes")

In [ ]:
readability_df = pd.DataFrame(readability_results[0], columns=["ari", "fog","fk", "linsear_write", "word_count"])
df = df.reset_index(drop=True)  # Ensure index alignment
readability_df = readability_df.reset_index(drop=True)
df[["ari", "fog", "fk", "linsear_write",  "word_count"]] = readability_df

In [ ]:
df = df[df.fog.notna()]
df.drop(['item_7', 'num_words','filing_date', 'period_of_report'], axis=1).to_csv("item7_multiple_readability.csv", index=False)